In [4]:
import json
import fractions
import re

def get_first_recipe(filename):
    with open(filename, 'r') as f:
        data = json.load(f)
        first_recipe = data[4199]
        print(first_recipe['name'])
        return first_recipe



def parse_quantity(quantity_str):
    quantity_str = quantity_str.replace('l', '').strip()  
    # Remove non-numeric parts of the string
    quantity_str = re.sub("[^\d\/\.]", "", quantity_str)
    
    if ',' in quantity_str:  # decimal quantities
        quantity = float(quantity_str.replace(',', '.'))
    elif '%' in quantity_str:  # percentage quantities
        quantity = float(quantity_str.replace('%', ''))*0.01
    elif 'cl' in quantity_str:  # centiliter quantities
        quantity = float(quantity_str.replace('cl', ''))*0.01
    elif 'g' in quantity_str:  # gram quantities
        quantity = float(quantity_str.replace('g', ''))*0.001
    elif 'kg' in quantity_str:  # kilogram quantities
        quantity = float(quantity_str.replace('kg', ''))
    elif 'ml' in quantity_str:  # milliliter quantities
        quantity = float(quantity_str.replace('ml', ''))*0.001
    elif 'l' in quantity_str:  # liter quantities
        quantity = float(quantity_str.replace('l', ''))
    elif 'tasse' in quantity_str:  # cup quantities
        quantity_str = quantity_str.replace('tasse', '')
        quantity = float(fractions.Fraction(quantity_str)) * 0.24
    elif '/' in quantity_str:  # fractions
        quantity = float(fractions.Fraction(quantity_str)) 
    else : 
        quantity = 0.1
    return quantity



def get_ingredient_costs(filename, recipe):
    with open(filename, 'r') as f:
        data = json.load(f)
        ingredient_names = [ingredient['name'] for ingredient in recipe['ingredients']] 
        ingredient_names = list(dict.fromkeys(ingredient_names))
        total_costs = {'USP': {'direct_price': 0, 'quantity_price': 0, 'kg_price': 0}, 
                       'ALDI': {'direct_price': 0, 'quantity_price': 0, 'kg_price': 0}}
        for ingredient_name in ingredient_names:
            quantity = next(ingredient['quantity'] for ingredient in recipe['ingredients'] if ingredient['name'] == ingredient_name) 
            quantity = parse_quantity(quantity)
            for source in ['USP', 'ALDI']:
                for item in data:
                    if item['ingredient'] == ingredient_name and item['source'] == source or ingredient_name in item.get('other_ingredients', []) and item['source'] == source:
                        item['price_per_kg'] = item['price_kg']
                        item['direct_price'] = item['price']
                        if item['price_kg'] != -1: 
                            if quantity == 0:  # Set a default quantity if none 
                                quantity = 0.1
                            item['quantity_price'] = quantity * item['price_kg']
                        else:
                            item['quantity_price'] = item['price']

                        print(f"Source : {item['source']}")
                        print(f"Ingredient original : {ingredient_name}")
                        print(f"Ingrédient : {item['ingredient']}")
                        print(f"Prix par kg : {item['price_per_kg']}")
                        print(f"Prix direct : {item['direct_price']}")
                        print(f"Prix selon quantité : {item.get('quantity_price', 'Non disponible')}")
                        print("-----------------------------")
                        
                        # add prices 
                        total_costs[source]['kg_price'] += item['price_per_kg']
                        total_costs[source]['direct_price'] += item['direct_price']
                        total_costs[source]['quantity_price'] += item.get('quantity_price', 0)
                        break


    for source in total_costs.keys():
        print(f"\nTotal cost for {source} ingredients:")
        print(f"Price per kg: {round(total_costs[source]['kg_price'],2)}")
        print(f"Direct price: {round(total_costs[source]['direct_price'],2)}")
        print(f"Price according to quantity: {round(total_costs[source]['quantity_price'],2)}")
        

        print("-----------------------------")


recipe_file = 'recipe_marmiton.json'
ingredient_file = 'items_ingredient.json'

recipe = get_first_recipe(recipe_file)
get_ingredient_costs(ingredient_file, recipe)


Profiteroles express
Source : ALDI
Ingredient original : chocolat noir
Ingrédient : cola
Prix par kg : -1
Prix direct : 0.59
Prix selon quantité : 0.59
-----------------------------
Source : USP
Ingredient original : beurre
Ingrédient : beurre
Prix par kg : 15.5
Prix direct : -1
Prix selon quantité : 1.55
-----------------------------
Source : ALDI
Ingredient original : beurre
Ingrédient : beurre
Prix par kg : 25.0
Prix direct : 9.99
Prix selon quantité : 2.5
-----------------------------
Source : USP
Ingredient original : lait
Ingrédient : lait
Prix par kg : 1.25
Prix direct : -1
Prix selon quantité : 0.125
-----------------------------
Source : ALDI
Ingredient original : lait
Ingrédient : lait
Prix par kg : -1
Prix direct : 2.99
Prix selon quantité : 2.99
-----------------------------
Source : USP
Ingredient original : vanille bien sûr, mais pourquoi pas châtaigne, caramel, crème brûlée... voire même, poire, figue, banane, tiramisu... Soyons fous !
Ingrédient : poire
Prix par kg : 60

In [ ]:
def get_ingredient_costs(filename, recipe):
    with open(filename, 'r') as f:
        data = json.load(f)
        ingredient_names = [ingredient['name'] for ingredient in recipe['ingredients']] 
        ingredient_names = list(dict.fromkeys(ingredient_names))
        total_costs = {'USP': {'direct_price': 0, 'quantity_price': 0, 'kg_price': 0}, 
                       'ALDI': {'direct_price': 0, 'quantity_price': 0, 'kg_price': 0}}
        for ingredient_name in ingredient_names:
            quantity = next(ingredient['quantity'] for ingredient in recipe['ingredients'] if ingredient['name'] == ingredient_name) 
            quantity = parse_quantity(quantity)
            for source in ['USP', 'ALDI']:
                for item in data:
                    if item['ingredient'] == ingredient_name and item['source'] == source or ingredient_name in item.get('other_ingredients', []) and item['source'] == source:
                        item['price_per_kg'] = item['price_kg']
                        item['direct_price'] = item['price']
                        if item['price_kg'] != -1: 
                            if quantity == 0:  # Set a default quantity if none 
                                quantity = 0.1
                            item['quantity_price'] = quantity * item['price_kg']
                        else:
                            item['quantity_price'] = item['price']

                        
                        # add prices 
                        total_costs[source]['kg_price'] += item['price_per_kg']
                        total_costs[source]['direct_price'] += item['direct_price']
                        total_costs[source]['quantity_price'] += item.get('quantity_price', 0)
                        break
    return total_costs




def get_all_recipes_cost(filename, recipe_file):
    all_recipes_costs = {}
    with open(recipe_file, 'r') as f:
        recipes = json.load(f)
        for i, recipe in enumerate((recipes)):
            print(f"Processing recipe #{i+1}: {recipe['name']}")
            all_recipes_costs[recipe['name']] = get_ingredient_costs(filename, recipe)
    return all_recipes_costs

def save_costs_to_file(filename, data):
    with open(filename, 'w') as f:
        json.dump(data, f, ensure_ascii=False, indent=4)

recipe_file = 'recipe_marmiton.json'
ingredient_file = 'items_ingredient.json'
output_file = 'recipe_costs.json'

all_recipes_costs = get_all_recipes_cost(ingredient_file, recipe_file)
save_costs_to_file(output_file, all_recipes_costs)